In [15]:
import pickle
import numpy as np
import datetime

In [8]:
df = pickle.load(open('/home/sarahwie/Documents/pubmed-nlp-research/C_article_replication_and_translational_classification/df.p', 'rb'))

In [9]:
#check labels- great
print df.columns.values
print df['label'].unique()

['pmid' 'title' 'abstract' 'qualifier_terms' 'descriptor_terms' 'label']
['T0' 'T1/T2' 'T3/T4' 'TX']


In [19]:
# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [50]:
df[0:10]

,pmid,title,abstract,qualifier_terms,descriptor_terms,label,boolean_label
0,23195993,Gene expression profiles in peripheral blood m...,Occupational exposure to nickel (Ni) is associ...,genetics metabolism epidemiology drug effects ...,Adult Asian Continental Ancestry Group Biomark...,T0,1
1,25077433,SOX2 is a cancer-specific regulator of tumour ...,Although the principles that balance stem cell...,genetics metabolism pathology metabolism patho...,"Animals Carcinoma, Squamous Cell Cell Line, Tu...",T0,1
2,24107601,Imaging and cerebrospinal fluid biomarkers in ...,The pathophysiological process of Alzheimer's ...,cerebrospinal fluid genetics radionuclide imag...,"Aged Aged, 80 and over Alzheimer Disease Amylo...",T0,1
3,24891029,Preliminary evidence of cognitive and brain ab...,To ascertain whether pediatric obesity without...,pathology physiopathology physiopathology,Adolescent Attention Brain Cognition Executive...,T0,1
4,21691448,Obese Adolescents with Type 2 Diabetes Mellitu...,The rates of type 2 diabetes (T2DM) continue t...,,,T0,1
5,22765222,Diversity of 5S rRNA genes within individual p...,We examined intragenomic variation of paralogo...,chemistry classification genetics chemistry ge...,"Bacteria DNA, Ribosomal Databases, Nucleic Aci...",T0,1
6,24401686,Myoinositol and glutamate complex neurometabol...,To obtain quantitative neurometabolite measure...,analogs & derivatives metabolism metabolism me...,Adult Aspartic Acid Brain Injuries Case-Contro...,T0,1
7,22914093,Antibiotics in early life alter the murine col...,Antibiotics administered in low doses have bee...,drug effects physiology administration & dosag...,Adiposity Age Factors Animals Anti-Bacterial A...,T0,1
8,23426830,Elevated serum anti-Müllerian hormone in adole...,Serum anti-Müllerian hormone (AMH) is linked t...,blood pathology ultrasonography blood ultrason...,Adolescent Anti-Mullerian Hormone Child Female...,T0,1
9,24344399,Association of obesity-mediated insulin resist...,The hypothalamus is important in hunger and me...,blood analysis anatomy & histology blood blood...,Adolescent Adult Brain-Derived Neurotrophic Fa...,T0,1


### Generate probssentences 

In [49]:
#add boolean labels
df['boolean_label'] = 1
for row in df.iterrows():
    if row[1]['label'] != 'T0':
        df.set_value(row[0], 'boolean_label', 0)

In [58]:
#not doing stratified but just shuffling to match method used for probssentences (movie reviews)
#do a larger split because have so little data
fracTrain = 0.80
nSamples = df.shape[0]
order = np.random.permutation(nSamples) # come up with a random ordering
splitIndex = int(np.round(nSamples*fracTrain))
train1 = df.ix[order[:splitIndex],:]
test1 = df.ix[order[splitIndex:],:]

In [55]:
print train1.shape
print test1.shape

(283, 7)
(71, 7)


In [57]:
train1[0:10]

,pmid,title,abstract,qualifier_terms,descriptor_terms,label,boolean_label
201,25689199,Cervical cancer screening in a sexually transm...,We examined whether a sexually transmitted dis...,methods diagnosis diagnosis,Adult Ambulatory Care Facilities Early Detecti...,T3/T4,0
281,23441057,PedsQL™ sickle cell disease module: feasibilit...,Sickle cell disease (SCD) is an inherited chro...,,"Adolescent Adult Anemia, Sickle Cell Child Chi...",T1/T2,0
76,22001663,Is pain catastrophizing a stable trait or dyna...,This study sought to clarify whether pain-rela...,epidemiology psychology psychology statistics ...,"Arthralgia Arthroplasty, Replacement, Knee Cat...",T0,1
20,22869155,Comprehensive search for Alzheimer disease sus...,To evaluate the association of risk and age at...,genetics mortality genetics genetics genetics ...,"Age of Onset Aged Aged, 80 and over Alzheimer ...",T0,1
89,21645996,Evidence for association of hyperprolinemia wi...,There are multiple genetic links between schiz...,deficiency blood complications drug therapy th...,1-Pyrroline-5-Carboxylate Dehydrogenase Adoles...,T0,1
139,22433978,Effect of escitalopram on insomnia symptoms an...,The aim of this study was to determine the eff...,administration & dosage therapeutic use drug t...,Adult African Americans Citalopram Double-Blin...,T1/T2,0
129,22718030,Whey protein supplementation does not affect e...,Little is known about the effects of different...,physiology physiology drug effects physiology ...,Adult Appetite Body Composition Dietary Protei...,T1/T2,0
206,24357348,Useful tests for hepatic encephalopathy in cli...,Hepatic encephalopathy (HE) is a serious compl...,methods methods diagnosis psychology,"Diagnosis, Computer-Assisted Electroencephalog...",T3/T4,0
175,23413249,The effect of pulse sequence parameters and co...,Both technical and pathophysiologic factors af...,diagnosis pathology diagnosis administration &...,Adult Aged Algorithms Astrocytoma Brain Brain ...,T1/T2,0
84,24025359,Combining histone deacetylase inhibitors with ...,In the present study we show that histone deac...,genetics pharmacology therapeutic use drug the...,Adenoviridae Animals Antineoplastic Agents Car...,T0,1


In [59]:
"""
docprob takes two lists
* docs: a list of documents, each of which is a list of sentences
* models: the candidate word2vec models (each potential class)

it returns the array of class probabilities.  Everything is done in-memory.
"""

def docprobsentences(docs, mods):
    # score() takes a list [s] of sentences here; could also be a sentence generator
    sentlist = [s for d in docs for s in d]
    # the log likelihood of each sentence in this review under each w2v representation
    llhd = np.array( [ m.score(sentlist, len(sentlist)) for m in mods ] )
    # now exponentiate to get likelihoods, 
    lhd = np.exp(llhd - llhd.max(axis=0)) # subtract row max to avoid numeric overload
    # normalize across models (stars) to get sentence-star probabilities
    #all this transposing business does is make it so the total probability of a word 
    #   equals 1 between the 2 arrays (positive prob and negative).
    #and the pandas data frame just puts everything into rows/columns format for easy viz
    prob = pd.DataFrame( (lhd/lhd.sum(axis=0)).transpose() )
    # and finally average the sentence probabilities to get the review probability
    prob["doc"] = [i for i,d in enumerate(docs) for s in d]
    return prob

In [35]:
a = datetime.datetime.now().replace(microsecond=0)

# ****** Split the labeled and unlabeled training sets into clean sentences
#
sentences_pos = []  # Initialize an empty list of sentences
sentences_neg = []  # Initialize an empty list of sentences

inxs_pos = np.where(train1["boolean_label"] == 1)[0].tolist()
inxs_neg = np.where(train1["boolean_label"] == 0)[0].tolist()

print "Parsing sentences from training set"
for inx in inxs_pos:
    review = train1["abstract"].iloc[inx]
    sentences_pos += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)

for inx in inxs_neg:
    review = train1["abstract"].iloc[inx]
    sentences_neg += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)    


# ****** Split the labeled and unlabeled training sets into clean sentences
#
sentences = []  # Initialize an empty list of sentences

print "Parsing sentences from training set"
for review in train1["abstract"]:
    sentences += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)
        
from gensim.models import Word2Vec
import multiprocessing

## create a w2v learner 
basemodel = Word2Vec(
    workers=multiprocessing.cpu_count(), # use your cores
    iter=3, # iter = sweeps of SGD through the data; more is better
    hs=1, negative=0 # we only have scoring for the hierarchical softmax setup
    )       
basemodel.build_vocab(sentences) 
        
from copy import deepcopy
models = [deepcopy(basemodel) for i in range(2)]
models[0].train(sentences_neg, total_examples=len(sentences_neg) )
models[1].train(sentences_pos, total_examples=len(sentences_pos) )

# read in the test set as a list of a list of words
docs = []
for review in test1["abstract"]:
    docs.append(KaggleWord2VecUtility.review_to_sentences(review, tokenizer))

print "scoring test set"
# get the probs (note we give docprob our test set plus the models)
#probs = docprob(docs,models).astype(object)
probssentencesArticle = docprobsentences(docs,models).astype(object)
        
b = datetime.datetime.now().replace(microsecond=0)
print(b-a)

Parsing sentences from training set
Parsing sentences from training set
scoring test set
0:00:02


In [21]:
print probssentencesArticle.shape

(1541, 3)


In [30]:
#print probssentencesArticle
print len(probssentencesArticle['doc'].unique())

177


In [29]:
pickle.dump(probssentencesArticle, open('./pickled_objects/probssentencesArticle.p', 'wb'))

### Do the same for probssentences_train

In [31]:
# read in the training set as a list of a list of words
docs_train = []
for review in train1["abstract"]:
    docs_train.append(KaggleWord2VecUtility.review_to_sentences(review, tokenizer))

In [36]:
probssentences_train_Article = docprobsentences(docs_train,models).astype(object)

In [37]:
print probssentences_train_Article.shape

(1507, 3)


In [40]:
print len(probssentences_train_Article['doc'].unique())

177


In [42]:
pickle.dump(probssentences_train_Article, open('./pickled_objects/probssentences_train_Article.p', 'wb'))

In [43]:
pickle.dump(test1, open('./pickled_objects/articleTest1.p', 'wb'))

In [44]:
pickle.dump(train1, open('./pickled_objects/articleTrain1.p', 'wb'))

In [45]:
print train1.shape
print test1.shape

(177, 7)
(177, 7)


In [47]:
train1[0:10]

,pmid,title,abstract,qualifier_terms,descriptor_terms,label,boolean_label
324,24556849,Emergence of mild cognitive impairment in late...,It is difficult to reliably detect the earlies...,prevention & control diagnosis etiology psycho...,Age Factors Alzheimer Disease Cognition Compar...,T3/T4,0
272,23688215,Recognition of famous names predicts cognitive...,The ability to recognize familiar people is im...,diagnosis physiopathology psychology physiopat...,"Aged Aged, 80 and over Alzheimer Disease Brain...",T0,1
240,21702704,Examining the aging process through the stress...,The aging process is marked by a series of tra...,psychology psychology psychology physiopathology,"Adaptation, Psychological Aged Aging Automobil...",TX,0
169,20461380,"A first in human study of SB-743921, a kinesin...","To determine the maximum-tolerated dose (MTD),...",pharmacology therapeutic use pharmacology ther...,"Adult Aged Aged, 80 and over Antineoplastic Ag...",T1/T2,0
158,21318565,A prospective open-label study of aripiprazole...,Fragile X syndrome (FXS) is the most common in...,drug effects administration & dosage adverse e...,Adolescent Adult Aggression Antipsychotic Agen...,T1/T2,0
97,21106989,"CP-690,550, a therapeutic agent, inhibits cyto...","The retrovirus, human T-cell-lymphotrophic vir...",drug effects metabolism pharmacology drug effe...,"Adult Animals Cell Line, Tumor Cell Proliferat...",T0,1
210,22686210,The relevancy of community-based methods: usin...,"The rates of obesity, diabetes, and heart dise...",epidemiology methods statistics & numerical da...,Adult Alaska Community-Based Participatory Res...,T3/T4,0
121,24623140,Cytosolic phospholipase A2 protein as a novel ...,The objective of this study was to investigate...,administration & dosage methods genetics admin...,Animals Butadienes Drug Delivery Systems Enzym...,T0,1
179,23756294,Utilization of a benchmarking database to info...,An integral part of NICHE (Nurses Improving Ca...,psychology,Aged Attitude of Health Personnel Benchmarking...,T3/T4,0
329,22613170,Frontal gray matter reduction after breast can...,Cognitive changes related to cancer and its tr...,adverse effects therapeutic use pathology drug...,Adult Aged Antineoplastic Combined Chemotherap...,T3/T4,0


### Kaggle Word2VecUtility

In [17]:
import re
import nltk

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from nltk.corpus import stopwords


class KaggleWord2VecUtility(object):
    """KaggleWord2VecUtility is a utility class for processing raw HTML text into segments for further learning"""

    @staticmethod
    def review_to_wordlist( review, remove_stopwords=False ):
        # Function to convert a document to a sequence of words,
        # optionally removing stop words.  Returns a list of words.
        #
        # 1. Remove HTML
        review_text = BeautifulSoup(review).get_text()
        #
        # 2. Remove non-letters
        review_text = re.sub("[^a-zA-Z]"," ", review_text)
        #
        # 3. Convert words to lower case and split them
        words = review_text.lower().split()
        #
        # 4. Optionally remove stop words (false by default)
        if remove_stopwords:
            stops = set(stopwords.words("english"))
            words = [w for w in words if not w in stops]
        #
        # 5. Return a list of words
        return(words)
    
    @staticmethod
    def review_to_words( review, remove_stopwords=False ):
        # Function to convert a raw review to a string of words
        # The input is a single string (a raw movie review), and 
        # the output is a single string (a preprocessed movie review)
        #
        # 1. Remove HTML
        review_text = BeautifulSoup(review).get_text() 
        #
        # 2. Remove non-letters        
        review_text = re.sub("[^a-zA-Z]", " ", review_text) 
        #
        # 3. Convert to lower case, split into individual words
        words = review_text.lower().split()                             
        #
        # 4. Optionally remove stop words (false by default)
        if remove_stopwords:
            stops = set(stopwords.words("english"))
            words = [w for w in words if not w in stops]   
        #
        # 6. Join the words back into one string separated by space, 
        # and return the result.
        return( " ".join( words ))   

    # Define a function to split a review into parsed sentences
    @staticmethod
    def review_to_sentences( review, tokenizer, remove_stopwords=False ):
        # Function to split a review into parsed sentences. Returns a
        # list of sentences, where each sentence is a list of words
        #
        # 1. Use the NLTK tokenizer to split the paragraph into sentences
        raw_sentences = tokenizer.tokenize(review.decode('utf8').strip())
        #
        # 2. Loop over each sentence
        sentences = []
        for raw_sentence in raw_sentences:
            # If a sentence is empty, skip it
            if len(raw_sentence) > 0:
                # Otherwise, call review_to_wordlist to get a list of words
                sentences.append( KaggleWord2VecUtility.review_to_wordlist( raw_sentence, \
                  remove_stopwords ))
        #
        # Return the list of sentences (each sentence is a list of words,
        # so this returns a list of lists
        return sentences